In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# シンプルなニューラルネットワークを作る
class SimpleMNISTNet(nn.Module):
    def __init__(self):
        super(SimpleMNISTNet, self).__init__()
        # 28x28=784ピクセル → 128 → 64 → 10クラス
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        # 画像を1次元に変換
        x = x.view(-1, 784)
        # 活性化関数ReLUを適用
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # 最後は活性化しない（CrossEntropyLossが内部でやる）
        x = self.fc3(x)
        return x

# モデルを作成
model = SimpleMNISTNet()
print(model)  # 構造を確認


SimpleMNISTNet(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# データの前処理（正規化）
transform = transforms.Compose([
    transforms.ToTensor(),  # 画像をTensorに変換
    transforms.Normalize((0.1307,), (0.3081,))  # 平均と標準偏差で正規化
])

# MNISTデータセットをダウンロード＆読み込み
train_dataset = datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

test_dataset = datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

# DataLoaderでバッチ処理
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# データの形を確認
images, labels = next(iter(train_loader))
print(f"Batch画像の形: {images.shape}")  # torch.Size([64, 1, 28, 28])
print(f"Batchラベルの形: {labels.shape}")  # torch.Size([64])


100%|██████████| 9.91M/9.91M [00:00<00:00, 61.8MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.71MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.7MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 3.10MB/s]

Batch画像の形: torch.Size([64, 1, 28, 28])
Batchラベルの形: torch.Size([64])


In [ ]:
import torch.optim as optim

# 損失関数（CrossEntropyLoss）
criterion = nn.CrossEntropyLoss()

In [ ]:
# オプティマイザー（SGD or Adam）
optimizer = optim.Adam(model.parameters(), lr=0.001)
# または
# optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 試しに1バッチだけ学習してみる
model.train()  # 学習モードに
images, labels = next(iter(train_loader))

In [ ]:
# 順伝播
outputs = model(images)
loss = criterion(outputs, labels)
print(f"初期Loss: {loss.item()}")

初期Loss: 2.307684898376465


In [ ]:
# 逆伝播
optimizer.zero_grad()  # 勾配をリセット
loss.backward()  # 勾配を計算
optimizer.step()  # パラメータを更新

In [ ]:
# もう一度同じバッチで計算
outputs = model(images)
loss = criterion(outputs, labels)
print(f"1回更新後のLoss: {loss.item()}")  # 少し下がったはず！

1回更新後のLoss: 2.202392578125
